# 최소신장트리

가중치 그래프에서 모든 정점들을 연결하는 간선들의 가중치의 합이 최소가 되는 트리를 찾는 문제<br><br>
- 신장 트리 : n개의 정점을 포함하는 무향 그래프에서 **n개의 정점과 n-1개의 간선**으로 구성된 트리<br>
- 최소 신장 트리 : 가중치 그래프에서 신장 트리를 구성하는 간선들의 가중치의 합이 최소인 신장 트리

## 프림 알고리즘

한 정점에 연결된 간선들 중 하나씩 선택하면서 최소 신장 트리를 만들어 가는 방식
1. 임의의 정점을 하나 선택해서 시작
2. 선택한 정점들과 인접하는 정점들 중에 최소 비용의 간선이 존재하는 정점을 선택
3. 모든 정점이 선택될 때까지 두번째 과정 반복

### heapq 사용

In [1]:
myedges = [
    (7, 'A', 'B'), (5, 'A', 'D'),
    (8, 'B', 'C'), (9, 'B', 'D'),
    (7, 'B', 'E'), (5, 'C', 'E'),
    (7, 'D', 'E'), (6, 'D', 'F'),
    (8, 'E', 'F'), (9, 'E', 'G'),
    (11, 'F', 'G')
]

In [2]:
from collections import defaultdict
from heapq import *

def prim(start_node, edges):
    mst = [] # 최소신장트리
    adjacent_edges = defaultdict(list) # 모든 간선 정보를 저장할 딕셔너리
    for weight, n1, n2 in edges:
        adjacent_edges[n1].append((weight, n1, n2))
        adjacent_edges[n2].append((weight, n1, n2))
        
    connected_nodes = set(start_node) # 연결된 노드의 집합. start_node는 바로 삽입.
    candidate_edge_list = adjacent_edges[start_node] # 간선리스트에 현재 노드와 연결된 간선들을 삽입.
    heapify(candidate_edge_list) # 최소 가중치를 가지는 간선부터 추출하기 위해, heapq 로 변경.
    
    # 간선 리스트에 더 이상 간선이 없을 때까지
    while candidate_edge_list:
        weight, n1, n2 = heappop(candidate_edge_list) # 최소 가중치를 가지는 노드 pop
        if n2 not in connected_nodes: # 해당 노드에 연결된 인접정점이 연결 노드 집합에 들어있지 않다면(사이클 방지)
            connected_nodes.add(n2) # 연결노드 집합에 해당 인접 정점을 더한다.
            mst.append((weight, n1, n2)) # mst에 해당 간선 정보를 삽입한다.
            
            for edge in adjacent_edges[n2]: # 해당 인접정점의 간선정보를 순회하는데,
                if edge[2] not in connected_nodes: # 인접정점에 연결된 간선 노드가 연결된 노드 집합에 없다면
                    heappush(candidate_edge_list, edge) # 간선리스트에 해당 간선 정보를 삽입
    return mst

In [3]:
prim('A', myedges)

[(5, 'A', 'D'),
 (6, 'D', 'F'),
 (7, 'A', 'B'),
 (7, 'B', 'E'),
 (8, 'B', 'C'),
 (9, 'E', 'G')]

## 프림 알고리즘 : heapq 없는 풀이

In [33]:
graph = {
    0: [(1, 32),(2, 31),(5, 60),(6, 51)],
    1: [(0, 32),(2, 21)],
    2: [(0, 31),(1, 21),(4, 46),(6, 25)],
    3: [(4, 34),(5, 18)],
    4: [(2, 46),(3, 34),(5, 40),(6, 51)],
    5: [(0, 60),(3, 18),(4, 40)],
    6: [(0, 51),(2, 25),(4, 51)]    
}

In [41]:
# G: 그래프, s: 시작 정점
def mst_prim(G, s):
    N = len(G)
    key = [float('inf')] * N # 가중치를 무한대로 초기화
    pi = [None]*N # 트리에서 연결될 부모 정점 초기화
    visited = [False]*N # 방문 여부 초기화
    key[s] = 0 # 시작 정점의 가중치를 0으로 설정
    
    for _ in range(N): # 정점의 개수만큼 반복
        min_idx = -1
        min = float('inf')
        for i in range(N):
            if not visited[i] and key[i] < min:
                min = key[i]
                min_idx = i
        visited[min_idx] = True # 최소 가중치 정점 방문처리
            
        for v, val in G[min_idx]: # 선택 정점의 인접한 정점에 대해서
            if not visited[v] and val < key[v]:
                key[v] = val # 가중치 갱신
                pi[v] = min_idx # 트리에서 연결될 부모 정점 갱신
    print(key)
    print(pi)

In [42]:
mst_prim(graph, 0)

[0, 21, 31, 34, 46, 18, 25]
[None, 2, 0, 4, 2, 3, 2]


## 크루스칼 알고리즘